In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('../../../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Experiments

#### Imports

In [ ]:
import numpy as np
import torch
import time
import pytorch_lightning as pl
from pytorch_lightning import loggers
import config
from utils.Visual import draw_plot

# 1. Hyperparameter Tuning

*see coresponding notebook!*

# 2. Adversarial Training

In [ ]:
##############################################################################################################
##############################################################################################################
##############################################################################################################
from utils.TrainHelper import TrainHelper, get_all_models_data
from attacks.pgd import ProjectedGradientDescent
from classification.models.M5 import M5PLModule

# define hyperparameters
hparams = {
    "batch_size": 64,
    "learning_rate": 3e-4,
    "weight_decay": 0.001,
    "lr_decay": 0.95,
    "epochs": 20
}

# define attack configs
attack_configs = [
                  {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["inf"], 
                                   "epsilon":[0.05], "num_iter": [2]
                                 },
                   "META": {"TITLE": "L_inf_eps=0.05_iters=2"}
                   },
                  {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["inf"], 
                                   "epsilon":[0.05], "num_iter": [3]
                                 },
                   "META": {"TITLE": "L_inf_eps=0.05_iters=3"}
                   },
                  ]

th = TrainHelper()
for dec in  [0.1, 1, 5]:
    hparams["weight_decay"] = dec
    th.run(M5PLModule, hparams, attack_configs, save_epochs = [100])
##############################################################################################################
##############################################################################################################
##############################################################################################################

In [ ]:
##############################################################################################################
##############################################################################################################
##############################################################################################################
from utils.TrainHelper import TrainHelper, get_all_models_data
from attacks.pgd import ProjectedGradientDescent
from classification.models.M5 import M5PLModule

# define hyperparameters
hparams = {
    "batch_size": 64,
    "learning_rate": 3e-4,
    "weight_decay": 0.1,
    "lr_decay": 0.95,
    "epochs": 30
}

# define attack configs
attack_configs = [
                  {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["inf"], 
                                   "epsilon":[0.1], "num_iter": [1]
                                 },
                   "META": {"TITLE": "L_inf_eps=0.1_iters=1"}
                   },
                  {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["inf"], 
                                   "epsilon":[0.1], "num_iter": [2]
                                 },
                   "META": {"TITLE": "L_inf_eps=0.1_iters=2"}
                   },
                  {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["inf"], 
                                   "epsilon":[0.1], "num_iter": [3]
                                 },
                   "META": {"TITLE": "L_inf_eps=0.1_iters=3"}
                   },
                  {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["inf"], 
                                   "epsilon":[0.1], "num_iter": [4]
                                 },
                   "META": {"TITLE": "L_inf_eps=0.1_iters=4"}
                   },
                  {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["inf"], 
                                   "epsilon":[0.05], "num_iter": [1]
                                 },
                   "META": {"TITLE": "L_inf_eps=0.05_iters=4"}
                   },
                      {"ATTACK_CLASS": ProjectedGradientDescent, 
                   "ATTACK_ARGS": {"norm":["inf"], 
                                   "epsilon":[0.03], "num_iter": [5]
                                 },
                   "META": {"TITLE": "L_inf_eps=0.03_iters=5"}
                   },
                  ]

th = TrainHelper()
models = th.run(M5PLModule, hparams, attack_configs, save_epochs = [100])
##############################################################################################################
##############################################################################################################
##############################################################################################################

In [ ]:
from utils.TrainHelper import TrainHelper, get_all_models_data, get_model_info

list(get_all_models_data()["M5"].keys())

sorted([ (m["final_val_acc"], m["path"]) for m in get_all_models_data()["M5"].values()])[-5:]

In [ ]:
print(get_model_info("M5_attack_ProjectedGradientDescent_v88.p"))

In [ ]:
sorted([ m["path"] for m in get_all_models_data()["M5"].values()])[-5:]

# 3. Robustness Comparison

In [ ]:
##############################################################################################################
##############################################################################################################
##############################################################################################################

from utils.RobustnessExperiment import RobustnessExperiment, load_experiment
from attacks.FGA_Batch import fast_gradient_attack
from attacks.pgd import ProjectedGradientDescent
from classification.models.M5 import M5PLModule
from classification.models.SpectrogramCNN import SpectrogramCNNPLModule



exp_config = [{
              "attack_fn": ProjectedGradientDescent, 
              "attack_arg": {"norm":["inf"], "epsilon": [0, 0.0001, 0.0002, 0.0005, 0.01, 0.02, 0.05, 0.1, 0.2, 0.5, 1]},
              "meta": {"key_result":"success_rate", "key_config":"epsilon", "title":"FGSM"}
             },
            {
            "attack_fn": ProjectedGradientDescent, 
            "attack_arg": {"norm":["2"], "epsilon": [0, 1, 2, 5, 10, 20, 50, 100, 200, 500, 1000]},
            "meta": {"key_result":"success_rate", "key_config":"epsilon", "title":"FGA L2"}
            },
            {
            "attack_fn": ProjectedGradientDescent, 
            "attack_arg": {"norm":["1"], "epsilon": [100, 200, 500, 1000, 2000, 5000, 1e4, 2e4, 5e4, 1e5]},
            "meta": {"key_result":"success_rate", "key_config":"epsilon", "title":"FGA L1"}
            }]

# uncomment this if you want to create a new experiment instead!
#experiment = RobustnessExperiment(exp_config, title="EVALUATION_STANDARD")
experiment = load_experiment(exp_folder = "0001_EVALUATION_STANDARD")

for m in models:
    experiment.run(m, M5PLModule)
##############################################################################################################
##############################################################################################################
##############################################################################################################

# Further Analyzis

### Show all evaluated models

In [ ]:
(experiment.show_evaluated_models())

### Which Model performs best on Robustness-Evaluation?

In [ ]:
experiment.show_best_models(metric = "success_rate", best_n=5, limit_eps=8)

### Print info of best model

In [ ]:
from utils.TrainHelper import TrainHelper, get_all_models_data

get_all_models_data()["M5"]["M5_attack_L_INF_e0.3_REGULARIZATION_TUNING_v158.p"]

### Compare

In [ ]:
experiment.compare(config_key = "epsilon", results_key = "success_rate", models=['M5_attack_L_INF_e0.3_REGULARIZATION_TUNING_v151.p',
                                                                                 "M5_attack_L_inf, eps=sample_u_0.01_0.3_v150.p",
                                                                                 "M5_attack_try_regularization_v94.p",
                                                                        'M5_attack_num_iters_v99.p'
                                                                                ])

In [ ]:
experiment.compare(config_key = "epsilon", results_key = "success_rate", models=['1592000682.p',
                                                                                 "M5_attack_ProjectedGradientDescent_v69_epoch_10.p",
                                                                                 "M5_attack_ProjectedGradientDescent_v69_epoch_20.p",
                                                                                 "M5_attack_ProjectedGradientDescent_v69_epoch_30.p",
                                                                                 "M5_attack_ProjectedGradientDescent_v69_epoch_40.p",
                                                                                 "M5_attack_ProjectedGradientDescent_v69.p",
                                                                                 "M5_attack_ProjectedGradientDescent_v89.p"

                                                                                ])